In [ ]:
# To use this, open anaconda, then open terminal and type: pip install opencv-python
# This program requires two files to use.

# press 'q' keyboard button to stop video

In [5]:
from keras.models import load_model
from keras.preprocessing.image import img_to_array
from keras.preprocessing import image
import cv2
import numpy as np

In [6]:
face_classifier=cv2.CascadeClassifier('emotion_model/haarcascade_frontalface_default.xml')
classifier = load_model('emotion_model/EmotionDetectionModel.h5')

In [7]:
class_labels=['Angry','Happy','Neutral','Sad','Surprise']
cap=cv2.VideoCapture(0)

In [8]:
R = 0
G = 255
B = 0
while True:
    ret,frame=cap.read()
    labels=[]
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    faces=face_classifier.detectMultiScale(gray,1.3,5)

    for (x,y,w,h) in faces:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(B,G,R),40)


        roi_gray=gray[y:y+h,x:x+w]
        roi_gray=cv2.resize(roi_gray,(48,48),interpolation=cv2.INTER_AREA)

        if np.sum([roi_gray])!=0:
            roi=roi_gray.astype('float')/255.0
            roi=img_to_array(roi)
            roi=np.expand_dims(roi,axis=0)

            preds=classifier.predict(roi)[0]
            label=class_labels[preds.argmax()]
            
            if label == 'Sad':
                R = 255
                G = 255
                B = 0
            elif label == 'Angry':
                R = 0
                G = 0
                B = 255
            elif label == 'Neutral':
                R = 0
                G = 255
                B = 0
            elif label == 'Surprise':
                R = 255
                G = 165
                B = 0
            
            label_position=(x+50,y-50)
            cv2.putText(frame,label,label_position,cv2.FONT_HERSHEY_SIMPLEX,2,(B,G,R),3)
        else:
            cv2.putText(frame,'No Face Found',(20,20),cv2.FONT_HERSHEY_SIMPLEX,2,(0,255,0),3)
    
    cv2.imshow('Emotion Detector',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()